In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
!pip install sentence_transformers==2.7.0

Mounted at /content/gdrive/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_n

In [2]:
%cd /content/gdrive/MyDrive/Colab_Notebooks/DSCI691/MedNLI

/content/gdrive/MyDrive/Colab_Notebooks/DSCI691/MedNLI


In [3]:
import math
from sentence_transformers import models, losses, datasets, SentenceTransformer, InputExample
import logging
from datetime import datetime
import pandas as pd
import torch

In [4]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [9]:
train_batch_size = 4
max_seq_length = 128
num_epochs = 2

In [12]:
# Load the pre-trained model (update this path to the correct one)
model_save_path_snli = "/content/gdrive/MyDrive/Colab_Notebooks/output/training_SNLI_v2_10epochs_bert-base-uncased-2024-06-04_19-26-36"
model = SentenceTransformer(model_save_path_snli)

In [21]:
# Load the MedNLI dataset
def load_mednli(file_path):
    return pd.read_json(file_path, lines=True)

train_df = load_mednli('mli_train_v1.jsonl')
dev_df = load_mednli('mli_dev_v1.jsonl')
test_df = load_mednli('mli_test_v1.jsonl')

In [22]:
# Preprocess the data
train_df = train_df.rename(columns={"sentence1": "sent1", "sentence2": "sent2", "gold_label": "label"})
dev_df = dev_df.rename(columns={"sentence1": "sent1", "sentence2": "sent2", "gold_label": "label"})
test_df = test_df.rename(columns={"sentence1": "sent1", "sentence2": "sent2", "gold_label": "label"})

In [23]:
# Filter out invalid labels
train_df = train_df[train_df['label'].isin(['entailment', 'neutral', 'contradiction'])]
dev_df = dev_df[dev_df['label'].isin(['entailment', 'neutral', 'contradiction'])]
test_df = test_df[test_df['label'].isin(['entailment', 'neutral', 'contradiction'])]

In [16]:
# Map labels to integers
label_mapping = {'entailment': 0, 'neutral': 1, 'contradiction': 2}
train_df['labels'] = train_df['label'].map(label_mapping)
dev_df['labels'] = dev_df['label'].map(label_mapping)
test_df['labels'] = test_df['label'].map(label_mapping)

In [17]:
# Convert DataFrame to InputExample
def convert_to_input_examples(df):
    return [InputExample(texts=[row['sent1'], row['sent2']], label=row['labels']) for idx, row in df.iterrows()]

train_samples = convert_to_input_examples(train_df)
dev_samples = convert_to_input_examples(dev_df)
test_samples = convert_to_input_examples(test_df)

In [24]:
# Create DataLoader
train_dataloader = datasets.NoDuplicatesDataLoader(train_samples, batch_size=train_batch_size)
dev_dataloader = datasets.NoDuplicatesDataLoader(dev_samples, batch_size=train_batch_size)
test_dataloader = datasets.NoDuplicatesDataLoader(test_samples, batch_size=train_batch_size)

In [25]:
# Define the loss function
train_loss = losses.MultipleNegativesRankingLoss(model)

In [26]:
 #Save path of the model after fine-tuning
model_save_path_mednli = (
    "/content/gdrive/MyDrive/Colab_Notebooks/output/training_MedNLI_v2_" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)

In [27]:
# Configure the training
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data for warm-up
logger.info("Warmup-steps: {}".format(warmup_steps))

In [28]:
# Fine-tune the model on MedNLI
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path=model_save_path_mednli,
    use_amp=False,  # Disable mixed precision training for CPU
)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2808 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2808 [00:00<?, ?it/s]